## Description: prepare the notebook for manual annotation

In this notebook, three elements are added to the previously generated dataset:
- check that the chapter and verse_num columns are integers
- sign_info column, based on a recent version of the function
- reconstructed_verse column which is a gcons version of the verse with brackets indicating the reconstructed sections
- the spatial_arg_type column for annotating goal, source, location, etc. 
- remove the rows which have "MN" in preposition_1 and no preposition_2

In [1]:
import pandas as pd

from itertools import chain

from tfob import TFOb,  get_dss

DSS = get_dss()

PATH = 'data/biblical_datasets/leviticus/annotation_df_history/Leviticus_dataset.csv' # Modify before executing the notebook

**Locating corpus resources ...**

Name,# of nodes,# slots / node,% coverage
scroll,1001,1428.81,100
lex,10450,129.14,94
fragment,11182,127.91,100
line,52895,27.04,100
clause,125,12.85,0
cluster,101099,6.68,47
phrase,315,5.10,0
word,500995,2.81,99
sign,1430241,1.00,100


In [2]:
#df_1 = pd.read_csv('../2_datasets/generate_datasets/data/biblical_datasets/Jeremiah_dataset.csv').fillna("")
df = pd.read_csv(PATH).fillna("")

In [3]:
# Make the values in "chapter" and "verse_num" integers
df['chapter'] = df['chapter'].astype(int)
df['verse_num'] = df['verse_num'].astype(int)

In [4]:
# Reorganise the dataset by chapter, verse_num and scroll

# Custom sorting logic for "scroll" column
df['scroll_order'] = df['scroll'].apply(lambda x: 0 if x == "MT" else 1)

# Sort by "scroll_order", "chapter", and "verse_num"
df = df.sort_values(by=['chapter', 'verse_num', 'scroll_order'])

# Drop the temporary 'scroll_order' column
df = df.drop(columns='scroll_order')

df = df.reset_index(drop=True)

In [5]:
pd.set_option("display.max_columns", None)

In [6]:
df.head(20)

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,gcons_clause,subject,complement,cmpl_lex,dir_he,cmpl_constr,cmpl_nt,cmpl_anim,cmpl_det,cmpl_indiv,cmpl_complex,dir_he_dss,sign_info,stem,tense,preposition_1,preposition_2,preposition_3,preposition_4,preposition_5,preposition_6,preposition_7,cmpl_translation,motion_type,comments
0,52535,QRB[,MT,Leviticus,1,2,JQRJB,DBR >L BNJ JFR>L W >MRT >LHM >DM KJ JQRJB MKM ...,KJ JQRJB MKM QRBN L JHWH,,L JHWH,L JHWH/,0.0,prep,pers,anim,det,nmpr,simple,,,hif,impf,L,,,,,,,,,
1,52550,QRB[,MT,Leviticus,1,2,TQRJBW,DBR >L BNJ JFR>L W >MRT >LHM >DM KJ JQRJB MKM ...,MN H BHMH MN H BQR W MN H Y>N TQRJBW >T QRBNKM,,MN H BHMH MN H BQR W MN H Y>N,MN H BHMH/ MN H BQR/ W MN H Y>N/,0.0,prep + prep + prep,,,det,subs,complex,,,hif,impf,MN,MN,MN,,,,,,,
2,1969253,QRB[,4Q25,Leviticus,1,2,JQRJB,DBR >L BNJ JFR>L W >MRT >LHM >DM KJ JQRJB MKM ...,None,None,,,,,,,,,,0.0,000 00 000 00000 0 0000 0000 000 00 00000 000 ...,hif,impf,,,,,,,,,,
3,1969268,QRB[,4Q25,Leviticus,1,2,TQRJBW,DBR >L BNJ JFR>L W >MRT >LHM >DM KJ JQRJB MKM ...,None,None,,,,,,,,,,0.0,000 00 000 00000 0 0000 0000 000 00 00000 000 ...,hif,impf,,,,,,,,,,
4,52561,QRB[,MT,Leviticus,1,3,JQRJBNW,>M <LH QRBNW MN H BQR ZKR TMJM JQRJBNW >L PTX ...,JQRJBNW,,no complement,no complement,,,,,,,,,,hif,impf,,,,,,,,,,
5,52566,QRB[,MT,Leviticus,1,3,JQRJB,>M <LH QRBNW MN H BQR ZKR TMJM JQRJBNW >L PTX ...,>L PTX >HL MW<D JQRJB >TW L RYNW L PNJ JHWH,,>L PTX >HL MW<D,>L PTX/ >HL/ MW<D/,0.0,prep,,,det,subs,complex,,,hif,impf,>L,,,,,,,,,
6,1969280,QRB[,4Q25,Leviticus,1,3,JQRJBNW,>M <LH QRBNW MN H BQR ZKR TMM JQRJBNW >L PTX >...,None,None,,,,,,,,,,0.0,00 000 00000 00 0 000 000 0010 0000000 00 000 ...,hif,impf,,,,,,,,,,
7,1969285,QRB[,4Q25,Leviticus,1,3,JQRJB,>M <LH QRBNW MN H BQR ZKR TMM JQRJBNW >L PTX >...,None,None,,,,,,,,,,0.0,00 000 00000 00 0 000 000 0010 0000000 00 000 ...,hif,impf,,,,,,,,,,
8,52596,QRB[,MT,Leviticus,1,5,HQRJBW,W CXV >T BN H BQR L PNJ JHWH W HQRJBW BNJ >HRN...,W HQRJBW BNJ >HRN H KHNJM >T H DM,BNJ >HRN H KHNJM,no complement,no complement,,,,,,,,,,hif,perf,,,,,,,,,,
9,52710,QRB[,MT,Leviticus,1,10,JQRJBNW,W >M MN H Y>N QRBNW MN H KFBJM >W MN H <ZJM L ...,JQRJBNW,,no complement,no complement,,,,,,,,,,hif,impf,,,,,,,,,,


In [7]:
df.columns

Index(['verb_id', 'lex', 'scroll', 'book', 'chapter', 'verse_num',
       'gcons_verb', 'gcons_verse', 'gcons_clause', 'subject', 'complement',
       'cmpl_lex', 'dir_he', 'cmpl_constr', 'cmpl_nt', 'cmpl_anim', 'cmpl_det',
       'cmpl_indiv', 'cmpl_complex', 'dir_he_dss', 'sign_info', 'stem',
       'tense', 'preposition_1', 'preposition_2', 'preposition_3',
       'preposition_4', 'preposition_5', 'preposition_6', 'preposition_7',
       'cmpl_translation', 'motion_type', 'comments'],
      dtype='object')

In [8]:
# This is the most recent sign_info function which uses the .rec (1 October 2024)

def sign_info(verb):
    sign_rec_words = []
    
    scroll = verb.to_scrolls.scroll[0]
    verse = TFOb.section([verb.book[0], verb.chapter[0], verb.verse[0]], DSS, scroll)
    
    for word in verse.to_words:
        signs = word.to_signs
        sign_rec = []
        for sign in signs:
            if sign.type[0] == "cons": # only keeps consonants
                if sign.rec[0]:
                    sign_rec.append("1")
                else:
                    sign_rec.append("0")
        if sign_rec:
            sign_rec_words.append("".join(sign_rec))
        
    return " ".join(sign_rec_words).strip()

In [9]:
# Updates the dataframe (change the old sign_info column to one containing the return of the new function)

def update_sign_info(row):
    if row['scroll'] != 'MT':
        return sign_info(TFOb(int(row['verb_id']), DSS))
    return row['sign_info']

# Apply the function to the dataframe
df['sign_info'] = df.apply(update_sign_info, axis=1)

In [10]:
# Initialize an empty list to collect mismatch information
mismatches = []

# Function to check the lengths of sign_info and gcons_verse
def check_length(row):
    if row['scroll'] != 'MT':
        if len(row['sign_info']) != len(row['gcons_verse']):
            mismatches.append(f"Mismatch found in row {row.name}: sign_info='{row['sign_info']}' (len={len(row['sign_info'])}), gcons_verse='{row['gcons_verse']}' (len={len(row['gcons_verse'])})")

# Apply the function to the dataframe
df.apply(check_length, axis=1)

# Check if there are any mismatches and print them
if mismatches:
    print("Mismatches found:")
    for mismatch in mismatches:
        print(mismatch)
else:
    print("No mismatches found.")

No mismatches found.


In [11]:
# The reconstruct_verse function adds a gcons version of the verse with brackets indicating the reconstructed sections

# Add a column with the verse plus brackets indicating reconstructed letters

def reconstruct_verse(sign_info, verse):
    rec_verse = []
    is_reconstructed = False

    for c_verse, c_sign in zip(verse, sign_info):
        if c_sign == "1" and not is_reconstructed:
            is_reconstructed = True
            rec_verse.append("[")
        if c_sign == "0" and is_reconstructed:
            is_reconstructed = False
            if rec_verse[-1] == " ":
                rec_verse.pop()
                rec_verse.extend(["]", " "])
            else:
                rec_verse.append("]")
        rec_verse.append(c_verse)

    if is_reconstructed:
        rec_verse.append("]")
    return "".join(rec_verse)


df["reconstructed_verse"] = ""

df["reconstructed_verse"] = df.apply(lambda row: reconstruct_verse(row.sign_info, row.gcons_verse) if row.book != "MT" else "", axis=1)

In [12]:
# Delete the DSS duplicated

verb_ids = df.verb_id.to_list()

duplicates = set([verb_id for verb_id in verb_ids if verb_ids.count(verb_id) > 1])


df = df[~((df.gcons_clause == "None") & (df.subject == "None") & (df.verb_id.isin(duplicates)))] # delete duplicates
df.reset_index(drop=True, inplace=True) # reset the index for AnnotEasy

In [13]:
df.columns

Index(['verb_id', 'lex', 'scroll', 'book', 'chapter', 'verse_num',
       'gcons_verb', 'gcons_verse', 'gcons_clause', 'subject', 'complement',
       'cmpl_lex', 'dir_he', 'cmpl_constr', 'cmpl_nt', 'cmpl_anim', 'cmpl_det',
       'cmpl_indiv', 'cmpl_complex', 'dir_he_dss', 'sign_info', 'stem',
       'tense', 'preposition_1', 'preposition_2', 'preposition_3',
       'preposition_4', 'preposition_5', 'preposition_6', 'preposition_7',
       'cmpl_translation', 'motion_type', 'comments', 'reconstructed_verse'],
      dtype='object')

In [14]:
# Add the spatial_arg_type column

df["spatial_arg_type"] = ""

In [15]:
df.columns

Index(['verb_id', 'lex', 'scroll', 'book', 'chapter', 'verse_num',
       'gcons_verb', 'gcons_verse', 'gcons_clause', 'subject', 'complement',
       'cmpl_lex', 'dir_he', 'cmpl_constr', 'cmpl_nt', 'cmpl_anim', 'cmpl_det',
       'cmpl_indiv', 'cmpl_complex', 'dir_he_dss', 'sign_info', 'stem',
       'tense', 'preposition_1', 'preposition_2', 'preposition_3',
       'preposition_4', 'preposition_5', 'preposition_6', 'preposition_7',
       'cmpl_translation', 'motion_type', 'comments', 'reconstructed_verse',
       'spatial_arg_type'],
      dtype='object')

In [16]:
# Remove the columns that have MN as preposition 1 and not other prepositions

In [19]:
# have a look at the data
df_test = df[(df.preposition_1 == "MN") & (df.preposition_2 == "MN")]
df_test.head(5)
#len(df_test)

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,gcons_clause,subject,complement,cmpl_lex,dir_he,cmpl_constr,cmpl_nt,cmpl_anim,cmpl_det,cmpl_indiv,cmpl_complex,dir_he_dss,sign_info,stem,tense,preposition_1,preposition_2,preposition_3,preposition_4,preposition_5,preposition_6,preposition_7,cmpl_translation,motion_type,comments,reconstructed_verse,spatial_arg_type
1,52550,QRB[,MT,Leviticus,1,2,TQRJBW,DBR >L BNJ JFR>L W >MRT >LHM >DM KJ JQRJB MKM ...,MN H BHMH MN H BQR W MN H Y>N TQRJBW >T QRBNKM,,MN H BHMH MN H BQR W MN H Y>N,MN H BHMH/ MN H BQR/ W MN H Y>N/,0.0,prep + prep + prep,,,det,subs,complex,,,hif,impf,MN,MN,MN,,,,,,,,,
13,52799,QRB[,MT,Leviticus,1,14,HQRJB,W >M MN H <WP <LH QRBNW L JHWH W HQRJB MN H TR...,W HQRJB MN H TRJM >W MN BNJ H JWNH >T QRBNW,,MN H TRJM >W MN BNJ H JWNH,MN H TWR/ >W MN BN/ H JWNH/,0.0,prep + prep,,,det,subs,complex,,,hif,perf,MN,MN,,,,,,,,,,
14,1966826,QRB[,4Q24,Leviticus,1,14,HQRJB,W >M MN H <WP <LH QRBNW L JHWH W HQRJB MN H TW...,W HQRJB MN H TWRJM >W MN BNJ H JWNH >T QRBNW,,MN H TWRJM >W MN BNJ H JWNH,,0.0,,,,,,,0.0,1 11 10 0 000 000 00000 0 0000 0 00011 11 1 11...,hif,perf,MN,MN,,,,,,,,,[W >M M]N H <WP <LH QRBNW L JHWH W HQR[JB MN H...,
15,1951888,QRB[,4Q17,Leviticus,1,14,HQRJB,W >M MN H <WP <LH QRBNW L JHWH W HQRJB MN H TR...,W HQRJB MN H TRJM >W MN BNJ H JWNH >T QRBNW,,MN H TRJM >W MN BNJ H JWNH,,0.0,,,,,,,0.0,1 11 11 1 111 111 11111 1 1111 0 00111 11 1 11...,hif,perf,MN,MN,,,,,,,,,[W >M MN H <WP <LH QRBNW L JHWH] W HQ[RJB MN H...,
122,55549,QRB[,MT,Leviticus,7,14,HQRJB,W HQRJB MMNW >XD M KL QRBN TRWMH L JHWH L KHN ...,W HQRJB MMNW >XD M KL QRBN TRWMH L JHWH,,MMNW M KL QRBN,MN MN KL/ QRBN/,0.0,prep + prep,,,det,subs,complex,,,hif,perf,MN,MN,,,,,,,,,,


In [20]:
len(df_test)

5

In [7]:
# Identify the rows to drop based on the condition
index_to_drop = df[(df.preposition_1 == "MN") & (df.preposition_2 == "")].index
index_to_drop_2 = df[(df.preposition_1 == "MN") & (df.preposition_2 == "MN") & (df.preposition_3 == "")].index
index_to_drop_3 = df[(df.preposition_1 == "MN") & (df.preposition_2 == "MN") & (df.preposition_3 == "MN") & (df.preposition_4 == "")].index

# Drop the rows by index
df = df.drop(index_to_drop)
df = df.drop(index_to_drop_2)
df = df.drop(index_to_drop_3)

# Reset the index
df = df.reset_index(drop=True)

In [8]:
# Verification
df[(df.preposition_1 == "MN") & (df.preposition_2 == "")]

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,sign_info,stem,...,cmpl_indiv,cmpl_complex,motion_type,spatial_arg_type,preposition_1,preposition_2,preposition_3,preposition_4,comments,reconstructed_verse


In [25]:
df.head(20)

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,gcons_clause,subject,complement,cmpl_lex,dir_he,cmpl_constr,cmpl_nt,cmpl_anim,cmpl_det,cmpl_indiv,cmpl_complex,dir_he_dss,sign_info,stem,tense,preposition_1,preposition_2,preposition_3,preposition_4,preposition_5,preposition_6,preposition_7,cmpl_translation,motion_type,comments,reconstructed_verse,spatial_arg_type
0,52535,QRB[,MT,Leviticus,1,2,JQRJB,DBR >L BNJ JFR>L W >MRT >LHM >DM KJ JQRJB MKM ...,KJ JQRJB MKM QRBN L JHWH,,L JHWH,L JHWH/,0.0,prep,pers,anim,det,nmpr,simple,,,hif,impf,L,,,,,,,,,,,
1,1969253,QRB[,4Q25,Leviticus,1,2,JQRJB,DBR >L BNJ JFR>L W >MRT >LHM >DM KJ JQRJB MKM ...,None,None,,,,,,,,,,0.0,000 00 000 00000 0 0000 1111 111 11 11111 111 ...,hif,impf,,,,,,,,,,,DBR >L BNJ JFR>L W >MRT [>LHM >DM KJ JQRJB MKM...,
2,1969268,QRB[,4Q25,Leviticus,1,2,TQRJBW,DBR >L BNJ JFR>L W >MRT >LHM >DM KJ JQRJB MKM ...,None,None,,,,,,,,,,0.0,000 00 000 00000 0 0000 1111 111 11 11111 111 ...,hif,impf,,,,,,,,,,,DBR >L BNJ JFR>L W >MRT [>LHM >DM KJ JQRJB MKM...,
3,52561,QRB[,MT,Leviticus,1,3,JQRJBNW,>M <LH QRBNW MN H BQR ZKR TMJM JQRJBNW >L PTX ...,JQRJBNW,,no complement,no complement,,,,,,,,,,hif,impf,,,,,,,,,,,,
4,52566,QRB[,MT,Leviticus,1,3,JQRJB,>M <LH QRBNW MN H BQR ZKR TMJM JQRJBNW >L PTX ...,>L PTX >HL MW<D JQRJB >TW L RYNW L PNJ JHWH,,>L PTX >HL MW<D,>L PTX/ >HL/ MW<D/,0.0,prep,,,det,subs,complex,,,hif,impf,>L,,,,,,,,,,,
5,1969280,QRB[,4Q25,Leviticus,1,3,JQRJBNW,>M <LH QRBNW MN H BQR ZKR TMM JQRJBNW >L PTX >...,None,None,,,,,,,,,,0.0,11 111 11111 00 0 000 000 000 0000000 11 111 1...,hif,impf,,,,,,,,,,,[>M <LH QRBNW] MN H BQR ZKR TMM JQRJBNW [>L PT...,
6,1969285,QRB[,4Q25,Leviticus,1,3,JQRJB,>M <LH QRBNW MN H BQR ZKR TMM JQRJBNW >L PTX >...,None,None,,,,,,,,,,0.0,11 111 11111 00 0 000 000 000 0000000 11 111 1...,hif,impf,,,,,,,,,,,[>M <LH QRBNW] MN H BQR ZKR TMM JQRJBNW [>L PT...,
7,52596,QRB[,MT,Leviticus,1,5,HQRJBW,W CXV >T BN H BQR L PNJ JHWH W HQRJBW BNJ >HRN...,W HQRJBW BNJ >HRN H KHNJM >T H DM,BNJ >HRN H KHNJM,no complement,no complement,,,,,,,,,,hif,perf,,,,,,,,,,,,
8,52710,QRB[,MT,Leviticus,1,10,JQRJBNW,W >M MN H Y>N QRBNW MN H KFBJM >W MN H <ZJM L ...,JQRJBNW,,no complement,no complement,,,,,,,,,,hif,impf,,,,,,,,,,,,
9,52772,QRB[,MT,Leviticus,1,13,HQRJB,W H QRB W H KR<JM JRXY B MJM W HQRJB H KHN >T ...,W HQRJB H KHN >T H KL,H KHN,no complement,no complement,,,,,,,,,,hif,perf,,,,,,,,,,,,


In [9]:
df.to_csv(PATH, index=False)